# Simulate

In [1]:
import ams

## Import and Setting the Verbosity Level

We first import the `ams` library.

In [2]:
import ams

In [3]:
ams.__version__

'0.6.5.post25.dev0+ga5fc571'

We can configure the verbosity level for logging (output messages) by passing a verbosity level (10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL) to the `stream_level` argument of `ams.main.config_logger()`. Verbose level 10 is useful for getting debug output.

The logging level can be altered by calling ``config_logger`` again with new ``stream_level`` and ``file_level``.

In [4]:
ams.config_logger(stream_level=20)

Note that the above ``ams.config_logger()`` is a shorthand to ``ams.main.config_logger()``.

If this step is omitted, the default `INFO` level (`stream_level=20`) will be used.

## Run Simulations

### Load Case

AMS support multiple input file formats, including AMS ``.xlsx`` file, MATPOWER ``.m`` file, PYPOWER ``.py`` file, and PSS/E ``.raw`` file.

Here we use the AMS ``.xlsx`` file as an example. The source file locates at ``$HOME/ams/ams/cases/ieee14/ieee14_opf.xlsx``.

In [5]:
sp = ams.load(ams.get_case('ieee14/ieee14_opf.xlsx'), default_config=True)

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14_opf.xlsx"...
Input file parsed in 0.4313 seconds.
System set up in 0.0021 seconds.


### Inspect Models and Routines

In AMS, ``model`` refers to the device model, and all models are registered to an OrderedDict ``models``.

In [6]:
sp.models

OrderedDict([('Summary', Summary (1 device) at 0x1590a8fd0),
             ('Bus', Bus (14 devices) at 0x15996c040),
             ('PQ', PQ (11 devices) at 0x15996cac0),
             ('PV', PV (4 devices) at 0x15996ce80),
             ('Slack', Slack (1 device) at 0x1599799d0),
             ('Shunt', Shunt (2 devices) at 0x159984400),
             ('Line', Line (20 devices) at 0x1599848b0),
             ('ESD1', ESD1 (0 devices) at 0x15998afa0),
             ('Area', Area (2 devices) at 0x15999d700),
             ('Region', Region (0 devices) at 0x15999dfa0),
             ('SFR', SFR (0 devices) at 0x159a54790),
             ('GCost', GCost (5 devices) at 0x159a54e80),
             ('SFRCost', SFRCost (0 devices) at 0x159a61520)])

One can inspect the detailed model data by converting it to a pandas DataFrame.

In [7]:
sp.Bus.as_df()

,idx,u,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,owner
uid,,,,,,,,,,,,,
0,1,1.0,BUS1,69.0,1.1,0.9,1.03000,0.000000,0,0,1,1,1
1,2,1.0,BUS2,69.0,1.1,0.9,1.01970,-0.027981,0,0,1,1,1
2,3,1.0,BUS3,69.0,1.1,0.9,1.00042,-0.060097,0,0,1,1,1
3,4,1.0,BUS4,69.0,1.1,0.9,0.99858,-0.074721,0,0,1,1,1
4,5,1.0,BUS5,69.0,1.1,0.9,1.00443,-0.064315,0,0,1,1,1
5,6,1.0,BUS6,138.0,1.1,0.9,0.99871,-0.109998,0,0,2,2,2
6,7,1.0,BUS7,138.0,1.1,0.9,1.00682,-0.084285,0,0,2,2,2
7,8,1.0,BUS8,69.0,1.1,0.9,1.01895,-0.024339,0,0,2,2,2
8,9,1.0,BUS9,138.0,1.1,0.9,1.00193,-0.127502,0,0,2,2,2


Similarly, in AMS, all supported routines are registered to an OrderedDict ``routines``.

In [8]:
sp.routines

OrderedDict([('DCPF', DCPF at 0x1590a8fa0),
             ('PFlow', PFlow at 0x159a6c670),
             ('ACOPF', ACOPF at 0x159b5f220),
             ('DCOPF', DCOPF at 0x159b9b1c0),
             ('ED', ED at 0x159b9b670),
             ('RTED', RTED at 0x159b9be50),
             ('UC', UC at 0x159bb1b50)])

### Solve Power Flow

In AMS, the AC power flow and DC power flow are solved by PYPOWER ``runpf()`` and ``rundcpf()`` functions, respectively.

In [9]:
sp.DCPF.run()

Setup model for DCPF
DCPF has no objective function.
DCPF model set up in 0.0011 seconds.
PYPOWER Version 5.1.4, 27-June-2018
 -- DC Power Flow

DCPF completed in 0.0033 seconds with exit code 0.


0

Inspect the generator power outputs and bus angles.

In [10]:
sp.DCPF.pg.v

array([0.4  , 0.4  , 0.3  , 0.35 , 0.787])

In [11]:
sp.DCPF.aBus.v

array([ 0.        , -0.02887178, -0.0641022 , -0.07743542, -0.06670094,
       -0.11334928, -0.08549152, -0.02403816, -0.12824676, -0.13308983,
       -0.12681192, -0.1339144 , -0.13779051, -0.16285214])

Solve the AC power flow.

In [12]:
sp.PFlow.run()

Setup model for PFlow
PFlow has no objective function.
PFlow model set up in 0.0012 seconds.
PYPOWER Version 5.1.4, 27-June-2018

Newton's method power flow converged in 3 iterations.

PFlow completed in 0.0091 seconds with exit code 0.


 -- AC Power Flow (Newton)



0

Inspect the generator power outputs, bus angles, and bus voltages.

In [13]:
sp.PFlow.pg.v

array([0.4       , 0.4       , 0.3       , 0.35      , 0.81427213])

In [14]:
sp.PFlow.aBus.v

array([ 0.        , -0.03078883, -0.06173455, -0.07696511, -0.06707345,
       -0.11262148, -0.08526268, -0.02687732, -0.12646404, -0.12942483,
       -0.12356406, -0.13042896, -0.13475262, -0.16547667])

In [15]:
sp.PFlow.vBus.v

array([1.03      , 1.03      , 1.01      , 1.01140345, 1.01725551,
       1.03      , 1.0224715 , 1.03      , 1.02176879, 1.01554207,
       1.01911515, 1.01740699, 1.01445023, 1.0163402 ])

### Solve ACOPF

In AMS, the ACOPF is solved by PYPOWER ``runopf()`` function.

In [16]:
sp.ACOPF.run()

Setup model for ACOPF
ACOPF model set up in 0.0015 seconds.
PYPOWER Version 5.1.4, 27-June-2018
 -- AC Optimal Power Flow



Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011


ACOPF completed in 0.2583 seconds with exit code 0.


Converged!


0

In [17]:
sp.ACOPF.pg.v

array([0.49996655, 0.10000065, 0.10000064, 0.10000065, 1.49809555])

In [18]:
sp.ACOPF.qg.v

array([0.14990061, 0.14999993, 0.09997174, 0.09993129, 0.08059299])

In [19]:
sp.ACOPF.aBus.v

array([ 0.        , -0.04662694, -0.11452421, -0.11585264, -0.09988879,
       -0.18205907, -0.16148797, -0.14627229, -0.19500474, -0.19788944,
       -0.19253337, -0.19878038, -0.20322846, -0.23228802])

In [20]:
sp.ACOPF.vBus.v

array([1.09999682, 1.08133094, 1.05126009, 1.05401909, 1.06111871,
       1.05714675, 1.06795632, 1.08056893, 1.06468318, 1.05600708,
       1.05321342, 1.04626446, 1.04476777, 1.055634  ])

### Solve DCOPF

In AMS, DCOPF and other routines are modeled and solved with CVXPY.

In [21]:
sp.DCOPF.run()

Setup model of DCOPF
DCOPF model set up in 0.0017 seconds.
DCOPF solved as optimal in 0.0055 seconds with exit code 0.


In [22]:
sp.DCOPF.pg.v

array([0.5   , 0.5   , 0.3685, 0.3685, 0.5   ])

One can specify the solver by passing the solver name to the ``solver`` argument of ``run()``.

The default CVXPV solver ``ECOS`` might be slow or incapable of solving large-scale problems.

In [23]:
sp = ams.load(ams.get_case('matpower/case_ACTIVSg2000.m'), default_config=True)

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case_ACTIVSg2000.m"...
Input file parsed in 0.4876 seconds.
The bus index is not continuous, adjusted automatically.
System set up in 0.8002 seconds.


In [24]:
sp.DCOPF.run(solver='ECOS')

Setup model of DCOPF
DCOPF model set up in 0.0094 seconds.
DCOPF solved as optimal in 29.8392 seconds with exit code 0.


One can specify the solver by passing the solver name to the ``solver`` argument of ``run()``.

In [25]:
sp.DCOPF.run(solver='GUROBI')

DCOPF model set up in 0.0129 seconds.


Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-21


DCOPF solved as optimal in 0.7355 seconds with exit code 0.
